In [2]:
import os
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2Tokenizer, GPT2Model

from read_data import *
from features import * 
from special_character import * 
from selected_variables import *
from surprisal import *



#### single file

In [2]:
"""file_path = "dundee_corpus/english/sb08ma1p.dat"

sf03ma1p = pd.read_csv(file_path, sep='\s+', skiprows=1,
                        names=['WORD', 'TEXT', 'LINE', 'OLEN', 'WLEN', 'XPOS', 'WNUM', 'FDUR', 'OBLP', 'WDLP', 'LAUN', 'TXFR'], 
                        encoding='windows-1252')

print(len(sf03ma1p))


file_path = "dundee_corpus/english/sb08ma2p.dat"
sb08ma2p = pd.DataFrame(columns = ['WORD', 'TEXT', 'LINE', 'OLEN', 'WLEN', 'XPOS', 'WNUM', 'FDUR', 'OBLP', 'WDLP', 'FXNO', 'TXFR'])
i = 0
with open(file_path, 'r', encoding='windows-1252') as file:
    for line in file:
      if i== 0 :
        i = i+1
        continue

      line = line.strip().split()
      sb08ma2p.loc[i] =  [line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11]]
      i = i+1

print(len(sb08ma2p))

file_path = "dundee_corpus/english/items/tx08wrdp.dat"

tx08 = pd.read_csv(file_path, sep='\s+',
                        names=['WORD', 'TEXT', 'LINE', 'OLEN', 'WLEN', 'XPOS', 'WNUM', 'FDUR', 'OBLP', 'WDLP', 'LAUN', 'TXFR', 'FF', 'LF'], 
                        encoding='windows-1252')

print(len(tx08))

def filter_and_remove_duplicates(words, fxon, txfr, wnum):
    result_words = []
    result_fxon = []
    result_txfr = []
    result_wnum = []
    prev_word = None

    for word, fxon_value, txfr_value, wnum_value in zip(words, fxon, txfr, wnum):
        if word != prev_word:
            result_words.append(word)
            result_fxon.append(int(fxon_value))
            result_txfr.append(int(txfr_value))
            result_wnum.append(int(wnum_value))
            prev_word = word

    return result_words, result_fxon, result_txfr, result_wnum

WORDS, FXON, TXFR, WNUM = filter_and_remove_duplicates(sb08ma2p['WORD'], sb08ma2p['FXNO'], sb08ma2p['TXFR'], sb08ma2p['WNUM'])
print("Filtered Words:", len(WORDS))
#print("Filtered FXON:", FXON)
#print("Filtered TXFR:", TXFR)
#print("Filtered WNUM:", len(WNUM))"""


'file_path = "dundee_corpus/english/sb08ma1p.dat"\n\nsf03ma1p = pd.read_csv(file_path, sep=\'\\s+\', skiprows=1,\n                        names=[\'WORD\', \'TEXT\', \'LINE\', \'OLEN\', \'WLEN\', \'XPOS\', \'WNUM\', \'FDUR\', \'OBLP\', \'WDLP\', \'LAUN\', \'TXFR\'], \n                        encoding=\'windows-1252\')\n\nprint(len(sf03ma1p))\n\n\nfile_path = "dundee_corpus/english/sb08ma2p.dat"\nsb08ma2p = pd.DataFrame(columns = [\'WORD\', \'TEXT\', \'LINE\', \'OLEN\', \'WLEN\', \'XPOS\', \'WNUM\', \'FDUR\', \'OBLP\', \'WDLP\', \'FXNO\', \'TXFR\'])\ni = 0\nwith open(file_path, \'r\', encoding=\'windows-1252\') as file:\n    for line in file:\n      if i== 0 :\n        i = i+1\n        continue\n\n      line = line.strip().split()\n      sb08ma2p.loc[i] =  [line[0], line[1], line[2], line[3], line[4], line[5], line[6], line[7], line[8], line[9], line[10], line[11]]\n      i = i+1\n\nprint(len(sb08ma2p))\n\nfile_path = "dundee_corpus/english/items/tx08wrdp.dat"\n\ntx08 = pd.read_csv(file_

### MA1P.DAT files

In [3]:
#len(result_df[result_df['Text File'] == '02'])
#print(len(result_df))
#filtered_df = result_df[(result_df['Text File'] == '03') & (result_df['Participant ID'] == 'sf') & (result_df['LINE'] == 3) & (result_df['TEXT'] == 8)]
#print(filtered_df)
#print(result_df[:7430])

In [14]:
directory = "dundee_corpus/english/"
result_df = read_data_ma1p(directory)
result_df2 = read_data_ma2p(directory)

In [19]:
#"""#result_df[7434:]"""
result_df2.isna().sum()
#result_df.isna().sum()


WORD              11
TEXT               0
LINE               0
OLEN               0
WLEN               0
XPOS               0
WNUM               0
FDUR               0
OBLP               0
WDLP               0
FXNO               0
TXFR               0
Participant ID     0
Text File          0
dtype: int64

In [4]:
# Step 2: Preprocess data
words, wnum, wlen, xpos, wdlp, oblp, fdur, words_to_ignore, participant_id, text_file, WORDs, FXNO, TXFR, WNUM, Participant_ID, Text_File = preprocess_data(result_df, result_df2)
len(text_file)
len(WORDs)


614991

In [ ]:


# Step 3: Clean text
cleaned_text_list = remove_special_characters(words, words_to_ignore)
cleaned_text_only = [cleaned_text for cleaned_text, special_char in cleaned_text_list]
special_char_only = [special_char for cleaned_text, special_char in cleaned_text_list]

# Step 4: Tokenize words into subwords
subwords, makeup_words = tokenize_words_into_subwords(words, words_to_ignore)
cleaned_subwords, cleaned_makeup_words = tokenize_words_into_subwords(cleaned_text_only, words_to_ignore)

# Step 5: Calculate subword information
num_subwords, subword_lengths, subword_infos = calculate_subword_info(subwords, wnum, words_to_ignore)
cl_num_subwords, cl_subword_lengths, cl_subword_infos = calculate_subword_info(cleaned_subwords, wnum, words_to_ignore)

# Step 6: Analyze gaze landing
fixation_letter, subpart_subword, fixation_position = gaze_landed_on_subwords(subwords, oblp, words_to_ignore)
cl_fixation_letter, cl_subpart_subword, cl_fixation_position = gaze_landed_on_subwords(cleaned_subwords, wdlp, words_to_ignore)


In [ ]:
def main():
    
    # Step 7: Create DataFrame
    dict1 = {'id': participant_id,'text_file': text_file, 'WORD': words, 'SBW': subwords,'WLEN': wlen, 
              'SWONUM': num_subwords, 'SWNUM': cl_num_subwords, 
             'SWOLEN': subword_lengths, 'SWLEN': cl_subword_lengths, 'SPCHAR': special_char_only,
             'SWIDX': subword_infos, 'XPOS': xpos, 'SWD': cl_subpart_subword,
             'SWDLP': cl_fixation_position, 'SWDLL': cl_fixation_letter, 'FDUR': fdur,}

    df1 = pd.DataFrame(dict1)
    return df1

# Call the main function to execute the processing steps and get the DataFrame
df1 = main()


In [ ]:
df1[7434:]

In [5]:
WORDS, FXON, TXFR, WNUM = filter_and_remove_duplicates(WORDs, FXNO, TXFR, WNUM,)
print("Filtered Words:", len(WORDS))
#print("Filtered FXON:", FXON)
#print("Filtered TXFR:", TXFR)
print("Filtered WNUM:", len(WNUM))

Filtered Words: 514980
Filtered WNUM: 514980


In [6]:
CL_WORD_list = remove_special_characters(WORDS, words_to_ignore)
CL_WORD_only = [cleaned_text for cleaned_text, special_char in CL_WORD_list]
Special_Char_only = [special_char for cleaned_text, special_char in CL_WORD_list]
CL_SUBWORDS, CL_MAKEUP_SUBWORD = tokenize_words_into_subwords(CL_WORD_only, words_to_ignore)
SUBWORDS, MAKEUP_SUBWORD = tokenize_words_into_subwords(WORDS, words_to_ignore)
#print("subwords:", SUBWORDS)

#print("subwords without any additional character :", len(CL_SUBWORDS))

#SWNUM, SWLEN = calculate_subword_info(CL_SUBWORDS, WNUM, words_to_ignore)

#print("Number of subwords:", len(SWNUM))
#print("Length of each subword:", len(SWLEN))
#print("Subword info:", subword1_infos)


In [7]:
dict3 = {'WORD': WORDS, 'SWORD': CL_SUBWORDS,  'TXFR': TXFR}            #, 'SWNUM': SWNUM, 'SWLEN': SWLEN,

df3 = pd.DataFrame(dict3)
#df3['MSWD'] = df3['SWNUM'].apply(lambda x: 1 if x > 1 else 0)
df3.head(50)

,WORD,SWORD,TXFR
0,Are,[Are],351
1,tourists,"[t, our, ists]",3
2,enticed,"[ent, iced]",1
3,by,[by],0
4,these,[these],73
5,attractions,"[att, ractions]",2
6,threatening,[threatening],3
7,their,[their],225
8,very,[very],56
9,existence?,[existence],4


In [3]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
file_path_template = "dundee_corpus/english/items/tx{:02d}wrdp.dat"
result_path_template = "C:\\Users\\ASUS\\Desktop\\Thesis\\surprisal\\results ({}).tsv"

combined_df = process_files_surprisal(tokenizer, file_path_template, result_path_template)
print(combined_df)

         subword       subw  surprisal  Text File
0            Are        Are  10.728434          1
1      Ġtourists  Ġtourists  14.185103          1
2           Ġent       Ġent  10.308628          1
3           iced       iced   0.410242          1
4            Ġby        Ġby   1.181995          1
...          ...        ...        ...        ...
63330       Ġare       Ġare   2.947925         20
63331   Ġpushing   Ġpushing  22.152325         20
63332     Ġahead     Ġahead   7.727186         20
63333       Ġtoo       Ġtoo  12.503385         20
63334          .          .  10.020133         20

[63335 rows x 4 columns]
